# Comunicação Interprocessual
***

Essa comunicação envolve mandar dados de um thread para outra

Vimos que pipes nos permite criar caminhos entre um arquivo e outro e mandar dados de um arquivo para o outro.

Entretanto, podemos usar pipes para mandar dados entre uma thread e outra, assim montando uma rede de comunicação entre as diferentes threads que estão sendo executadas

Existem dois tipos de pipes para esse fim: **pipe anônimos** e **pipe nomeados**.

Um **pipe anônimo** é aquele que é criado instantaneamente, em execução do programa, enquanto que o **nomeado** é baseado num arquivo pré-existente ou recentemente criado, qualquer thread tem acesso a esse arquivo.

**pipeout**: quem manda os dados e **pipein** quem recebe os dados

**sockets** também é um tipo de comunicação interprocessual

***
### Pipe Anônimos
***

In [1]:
import os
import time
from threading import Thread

In [2]:
# Manda dados da thread filho para thread pai
def son(pipeout):
    zzz = 0
    while True:
        time.sleep(zzz)
        msg = ('Spam {0:03d}'.format(zzz)).encode()
        # Escrever a mensagem no pipeout
        os.write(pipeout, msg)
        # Garantir que essa variavel não tenha valor maior que 5
        zzz = (zzz + 1) % 5

In [3]:
def father(pipein):
    while True:
        # Lê 32 bytes do que ta escrito no pipein
        line = os.read(pipein, 32)
        print("Pai {0} recebeu [{1}] às {2}".format(os.getpid(), line, time.time()))

In [4]:
# Inicializa os pipes
pipein, pipeout = os.pipe()

In [5]:
# Cria e rodo a Thread filho e passo o pipeout para ele
thread = Thread(target=son, args=(pipeout,))
thread.start()

In [6]:
# Executo a thread pai
father(pipein)

Pai 29975 recebeu [b'Spam 003'] às 1500674280.5707715
Pai 29975 recebeu [b'Spam 004Spam 000'] às 1500674284.5716484
Pai 29975 recebeu [b'Spam 001'] às 1500674285.572737
Pai 29975 recebeu [b'Spam 002'] às 1500674287.5733163
Pai 29975 recebeu [b'Spam 003'] às 1500674290.5763717
Pai 29975 recebeu [b'Spam 004Spam 000'] às 1500674294.5777493
Pai 29975 recebeu [b'Spam 001'] às 1500674295.578826
Pai 29975 recebeu [b'Spam 002'] às 1500674297.5809207
Pai 29975 recebeu [b'Spam 003'] às 1500674300.5819018
Pai 29975 recebeu [b'Spam 004Spam 000'] às 1500674304.5860007
Pai 29975 recebeu [b'Spam 001'] às 1500674305.5870888


KeyboardInterrupt: 

***
### Pipe Nomeados
***

In [6]:
import os
import time
import sys

In [7]:
# fifo nome do arquivo gerando para criar um pipe (file in file out)
fifoname = os.path.join(os.getcwd(), 'tmp', 'pipefifo')

In [8]:
def son():
    # Abre o arquivo no modo de escrita, não pode ler
    pipeout = os.open(fifoname, os.O_WRONLY)
    zzz = 0
    while True:
        time.sleep(zzz)
        msg = ('Spam {0:03d}\n'.format(zzz)).encode()
        os.write(pipeout, msg)
        zzz = (zzz + 1) % 5

In [9]:
def father():
    pipein = open(fifoname)
    while True:
        line = pipein.readline()
        print("Pai {0} recebeu {1} às {2}".format(os.getpid(), line, time.time()))

In [10]:
if not os.path.exists(fifoname):
    # Cria o arquivo fifoname, porém tem que criar a pasta tmp
    os.mkfifo(fifoname)
if len(sys.argv) == 1:
    father()
else:
    son()

FileNotFoundError: [Errno 2] No such file or directory